<a href="https://colab.research.google.com/github/Sri-Pooja00/2203A51341_NLP/blob/main/2203A51341_08_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:

import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from torch.utils.data import Dataset, DataLoader
from transformers import Trainer, TrainingArguments

# Initialize the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Define example text data
text_data = """Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, who lived in the woods. One day, her mother asked her to take a basket of goodies to her grandmother. On her way through the woods, she met a big bad wolf who wanted to eat her."""

tokenizer.pad_token = tokenizer.eos_token

# Tokenize the text data
tokens = tokenizer(text_data, return_tensors='pt', max_length=512, truncation=True, padding="max_length")


class TextDataset(Dataset):
    def __init__(self, tokens):
        self.tokens = tokens

    def __len__(self):
        return self.tokens["input_ids"].size(0)

    def __getitem__(self, idx):
        return {
            "input_ids": self.tokens["input_ids"][idx],
            "attention_mask": self.tokens["attention_mask"][idx],
            "labels": self.tokens["input_ids"][idx]
        }

# Load data into the dataset
dataset = TextDataset(tokens)


In [8]:

import os
os.environ["WANDB_DISABLED"] = "true"
def train_model(epochs):
    # Define training arguments
    training_args = TrainingArguments(
        output_dir="./results",
        overwrite_output_dir=True,
        num_train_epochs=epochs,
        per_device_train_batch_size=1,
        save_steps=10,
        save_total_limit=1,
        logging_dir="./logs",
        logging_steps=10
    )

    # Trainer to manage the training loop
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset
    )

    # Train the model
    trainer.train()

# Train the model with different epochs
for epochs in [20, 60, 70]:
    print(f"\nTraining model with {epochs} epochs...")
    train_model(epochs)



Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).



Training model with 20 epochs...


Step,Training Loss
10,1.876600
20,0.151800


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).



Training model with 60 epochs...


Step,Training Loss
10,0.040000
20,0.006600
30,0.006400
40,0.001900
50,0.002200
60,0.002300


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).



Training model with 70 epochs...


Step,Training Loss
10,0.004600
20,0.002800
30,0.003700
40,0.001100
50,0.001400
60,0.001000
70,0.002900


In [9]:
def generate_text(seed_text):
    # Encode the input text and set the pad token
    input_ids = tokenizer.encode(seed_text, return_tensors="pt", padding=True)
    model.config.pad_token_id = model.config.eos_token_id

    # Generate text
    output = model.generate(
        input_ids,
        max_length=100,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        top_k=50,
        top_p=0.95,
        attention_mask=(input_ids != model.config.pad_token_id)  # Handle attention mask
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Example generation
seed_text = "Once upon a time"
generated_text = generate_text(seed_text)
print("\nGenerated Text:", generated_text)


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Generated Text: Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, who lived in the woods. One day, her mother asked her to take a basket of goodies to her aunt. On her way through the forest, she met a big bad wolf who wanted to eat her.
